In [13]:
from keras import backend as K

if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

import keras.backend.tensorflow_backend 
from keras.backend.tensorflow_backend import set_session

import keras, os, glob
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
# from sklearn.model_selection import train_test_split
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam


train_dir = 'C:\\Users\\ADSTEC\\DLTEST\\cifar10\\IVI_cifar10' # 学習データ置き場
train_folder = os.listdir(train_dir)

test_dir = 'C:\\Users\\ADSTEC\\DLTEST\\cifar10\\IVI_cifar10_test' # 評価データ置き場

image_size_x = 32 # 元画像サイズX
image_size_y = 32 # 元画像サイズY

X_train = [] # 学習に使う画像データの格納先
Y_train = [] # 学習に使うラベルデータの格納先
X_test = [] # 評価に使う画像データの格納先
Y_test = [] # 評価に使うラベルデータの格納先
Y_test_index = 1 # airplaneは１

# 学習ファイル用データ抜き出し
for index, name in enumerate(train_folder):
    dir = "./IVI_cifar10/" + name
    train_files = glob.glob(dir + "/*")
    for i, file in enumerate(train_files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size_x, image_size_y))
        data = np.asarray(image)
        X_train.append(data) # 学習画像データをnumpy形式に変換
        Y_train.append(index) #　学習画像のラベルデータ(リストデータ)

# 評価ファイル用データ抜き出し
test_files = glob.glob(test_dir + "/*")
for i, file in enumerate(test_files):
    image = Image.open(file)
    image = image.convert("RGB")
    image = image.resize((image_size_x, image_size_y))
    data = np.asarray(image)
    X_test.append(data) # 評価画像データをnumpy形式に変換
    Y_test.append(Y_test_index) # 評価画像のラベルデータ(リストデータ)

# 学習データ、評価データをアレイデータに変換
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

#  学習画像データ、評価画像データの各ピクセルを255で割り0-1のデータで表示する
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 学習用ラベルデータ、評価用ラベルデータをリストからベクトル変換
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

num_classes = 10 # 分類項目数
model = Sequential() # Kerasのモデル指定

# 以下学習レイヤー設定

# CNN layer 1
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary()) # モデルのサマリーを表示

model.compile(
loss='categorical_crossentropy', # 損失関数の設定
optimizer=Adam(lr=0.001), # 最適化法の指定
metrics=['accuracy'])

fit = model.fit(X_train, Y_train,
              batch_size=128,
              epochs=40,
              verbose=1,
              validation_split=0.1 # 今回は訓練データセットの10%をvalidationデータセットとして使う
                )

score = model.evaluate(X_test, Y_test,
                    verbose=0
                    )

print('Test score:', score[0])
print('Test accuracy:', score[1])

folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)

model.save(os.path.join(folder, 'my_model.h5'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
__________

Test score: 2.2313846278525067
Test accuracy: 0.2703406813686979
